In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pff = pd.read_csv('pff.csv', index_col=0)
pff.head(2)

In [48]:
weeks = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']
pff = pff[pff['Week'].isin(weeks)]

In [49]:
pff['Week'] = pff['Week'].astype(int)
pff['Date'].map(lambda x : x[:2] < '05').value_counts()

False    7008
True      160
Name: Date, dtype: int64

In [51]:
# fix date to be in YYYY-MM-DD format
pff['Date'] = pff['Date'].map(lambda x : datetime.datetime.strptime(x, '%m/%d/%Y').isoformat()[:10])

In [52]:
# create pff home and pff away and sort so that way games line up
pff_home = pff[pff['H/A']=='H'].sort_values(by=['Season','Week','Date','Time','Team']).reset_index().reset_index().drop(columns='index').rename(columns={'level_0':'gamenum'})
pff_away = pff[pff['H/A']=='A'].sort_values(by=['Season','Week','Date','Time','Opposing Team']).reset_index().reset_index().drop(columns='index').rename(columns={'level_0':'gamenum'})

# check
test_match = pd.concat([pff_home[['gamenum', 'Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time']],
          pff_away[['gamenum', 'Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time']]],
         axis=1, ignore_index=True)

test_match.columns = ['gamenum1','Home_Team1','Away_Team1','Season1','Week1','Date1','Time1',
                  'gamenum2','Away_Team2','Home_Team2','Season2','Week2','Date2','Time2']

# this should be blank
test_match[test_match['Home_Team1'] != test_match['Home_Team2']]

,gamenum1,Home_Team1,Away_Team1,Season1,Week1,Date1,Time1,gamenum2,Away_Team2,Home_Team2,Season2,Week2,Date2,Time2


In [53]:
pff_fixed = pd.concat([pff_home,pff_away], axis=0).sort_values(by=['gamenum','H/A'])
pff_fixed

,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
0,0,Miami Dolphins,Pittsburgh Steelers,2006,1,2006-09-07,8:30pm,A,L,17,28,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-
0,0,Pittsburgh Steelers,Miami Dolphins,2006,1,2006-09-07,8:30pm,H,W,28,17,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-
1,1,Atlanta Falcons,Carolina Panthers,2006,1,2006-09-10,1:00pm,A,W,20,6,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-
1,1,Carolina Panthers,Atlanta Falcons,2006,1,2006-09-10,1:00pm,H,L,6,20,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-
2,2,New Orleans Saints,Cleveland Browns,2006,1,2006-09-10,1:00pm,A,W,19,14,68.4,66.9,58.6,86.2,72.0,50.4,66.0,67.4,81.6,61.8,59.4,57.8,-,66.3,60.9,57.2,68.7,60.7,58.8,57.0,70.5,71.8,71.9,53.2,68.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,3581,Los Angeles Rams,Arizona Cardinals,2019,17,2019-12-29,4:25pm,H,W,31,24,70.8,72.9,67.5,75.5,76.9,63.9,54.0,61.5,86.1,71.9,60.6,53.3,64.8,66.0,67.9,69.0,76.2,67.8,60.9,48.2,59.6,69.8,61.6,59.6,51.6,62.8
3582,3582,Philadelphia Eagles,New York Giants,2019,17,2019-12-29,4:25pm,A,W,34,17,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8
3582,3582,New York Giants,Philadelphia Eagles,2019,17,2019-12-29,4:25pm,H,L,17,34,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6
3583,3583,San Francisco 49Ers,Seattle Seahawks,2019,17,2019-12-29,8:20pm,A,W,26,21,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9


In [56]:
pff_fixed.to_csv('pff_pi.csv')